In [1]:
from datasets import load_dataset
from datasets import DatasetDict
import torch

dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_test_split = dataset['train'].train_test_split(test_size=0.13)

dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

dataset


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 14274
    })
    validation: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 2133
    })
})

In [2]:
from transformers import GPTNeoForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").to(device)



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

In [3]:
def prepare_data_for_inference(examples):
    # Format the input for BioGPT
    inputs = [f"Q: {q}" for q in examples['Question']]
    return inputs

# Prepare the validation dataset
validation_questions = prepare_data_for_inference(dataset['validation'])


In [4]:
def generate_baseline_predictions(model,tokenizer,questions):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    i=0
    for question in questions:
        inputs = tokenizer(f"{question}", return_tensors="pt").to(device)
        outputs = model.generate(inputs['input_ids'], max_length=70)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         print(response)
#         print('\n')
        predictions.append(response)
    
    return predictions

# Generate predictions for the validation dataset
predictions = generate_baseline_predictions(model,tokenizer,validation_questions)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask

In [5]:
!pip install evaluate
!pip install rouge-score
import evaluate
rouge = evaluate.load("rouge")
references = dataset['validation']['Answer']

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b55f2bae012a212afb6ea04be7a13b43f7b138c9a65c7c547bdb74ede0021b2b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [6]:
scores = rouge.compute(predictions=predictions, references=references)
print(scores)

{'rouge1': 0.1978090755850831, 'rouge2': 0.06200112370136836, 'rougeL': 0.1462159360530075, 'rougeLsum': 0.14937461082738251}


In [7]:
bleu = evaluate.load("bleu")
scores = bleu.compute(predictions=predictions, references=references)
print(scores)

{'bleu': 0.0034036375176860457, 'precisions': [0.420066407557077, 0.1196993532599196, 0.054316171952229556, 0.02854185957840841], 'brevity_penalty': 0.03622329961667256, 'length_ratio': 0.231585870347042, 'translation_length': 116553, 'reference_length': 503282}


In [8]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({
    'predictions': predictions,
    'references': references
})

# Save as a CSV file
df.to_csv('/kaggle/working/predictions_references.csv', index=False)